In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

import pandas as pd
from loguru import logger

logger.enable("probafcst")
logger.remove()
logger.add(sink=sys.stderr, level="INFO")

In [ ]:
data = pd.read_parquet("../data/bikes.parquet").asfreq("D").dropna()
y = data["bike_count"]
X = data.drop(columns=["bike_count"])
y

In [ ]:
from sklearn.model_selection import train_test_split
from sktime.forecasting.base import ForecastingHorizon

from probafcst.models.xgboost import XGBQuantileForecaster

quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=30, shuffle=False)
lags = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28]


fh = ForecastingHorizon(y_test.index, is_relative=False)
xgb_kwargs = dict(
    n_jobs=-1,
    # early_stopping_rounds=50,
    n_estimators=500,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
)
model = XGBQuantileForecaster(
    lags=lags,
    quantiles=quantiles,
    include_seasonal_dummies=True,
    cyclical_encodings=True,
    xgb_kwargs=xgb_kwargs,
)
model.fit(y_train, X_train)
model

In [ ]:
from probafcst.models.lgbm import LGBMQuantileForecaster

model = LGBMQuantileForecaster(
    lags=[1, 2], quantiles=quantiles, lgbm_kwargs=dict(verbose=0, n_estimators=10)
)

model.fit(y_train, X_train)
y_pred = model.predict_quantiles(fh, X, alpha=quantiles)

In [ ]:
# model.model.best_iteration, model.model.best_score

In [ ]:
from probafcst.utils.time import get_current_wednesday

get_current_wednesday()

In [ ]:
feature_importances = pd.Series(
    model.model.feature_importances_, index=model.feature_names_in_
)
feature_importances.nlargest(10).sort_values().plot(kind="barh")

In [ ]:
from sktime.performance_metrics.forecasting.probabilistic import (
    ConstraintViolation,
    EmpiricalCoverage,
    PinballLoss,
)

In [ ]:
fh = ForecastingHorizon(y_test.index, is_relative=False)
y_pred = model.predict_quantiles(fh, X, alpha=quantiles)

pinball_loss = PinballLoss()(y_test, y_pred)
empirical_coverage = EmpiricalCoverage()(y_test, y_pred)
constraint_violation = ConstraintViolation()(y_test, y_pred)
print(f"Pinball loss: {pinball_loss}")
print(f"Empirical coverage: {empirical_coverage}")
print(f"Constraint violation: {constraint_violation}")

In [ ]:
from probafcst.backtest import backtest, get_window_params

window_params = get_window_params(
    3, step_length_days=90, forecast_steps_days=7, freq="D"
)
results = backtest(
    model,
    y,
    **window_params,
    X=X,
    backend="loky",
    splitter_type="sliding",
    quantiles=quantiles,
)

In [ ]:
# import shap

# explainer = shap.TreeExplainer(
#     model.model, feature_names=model.feature_names_in_, data=features
# )
# features, labels = create_lagged_features(
#     X, y["bike_count"], lags=lags, include_seasonal_dummies=True, is_training=True
# )
# features = features.loc[y_test.index]
# labels = labels.loc[y_test.index]
# features.head()

In [ ]:
model.feature_names_in_

In [ ]:
# shap_values = explainer(features, labels)

In [ ]:
# averaged = shap_values.values.mean(axis=2)  # noqa: PD011
# averaged.shape

In [ ]:
# shap.summary_plot(averaged, features)